In [90]:
import itertools, re
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

In [91]:
Logging.setup_logging()

<Logger PySpice (INFO)>

In [92]:
def empty_ir(title="Candidate"):
    return {"title": title, "nodes": ["in","out","0"], "components": []}

In [93]:
def add_vin_and_load(ir, ac=1.0, Rload=1e2):
    # AC=1 V small-signal source recorded in IR
    ir["components"].append({"kind":"VAC","name":"in", "pos":"in","neg":"0","ac": ac})
    # Tiny "wire" placeholder for in->out path (a rewire anchor for series insertions)
    ir["components"].append({"kind":"R","name":"Rpath","n1":"in","n2":"out","value":1e-3})
    # Output load
    ir["components"].append({"kind":"R","name":"Rload","n1":"out","n2":"0","value": Rload})

In [94]:
from PySpice.Spice.Netlist import SubCircuitFactory

class RC_LowPass(SubCircuitFactory):
    NAME  = 'rc_lp'
    NODES = ('n1', 'n2', 'n3')
    def __init__(self, R=1@u_kΩ, C=1@u_uF):
        super().__init__()
        self.R(1, 'n1', 'n2', R)
        self.C(1, 'n2', 'n3',  C)

In [95]:
SUBCKT_REGISTRY = {
    'rc_lp':  RC_LowPass
}

In [96]:
def emit_pyspice(ir: dict, subckt_registry) -> Circuit:
    
    c = Circuit(ir["title"])

    for comp in ir["components"]:
        k = comp["kind"].upper()
        if k == "R":
            n1 = comp["n1"]; n2 = comp["n2"]
            c.R(comp["name"], n1, n2, comp["value"])

        elif k == "VAC":   # AC source for .ac analysis
            ac_amp = float(comp["ac"])
            #src = c.V(comp["name"], comp["pos"], comp["neg"], amplitude=ac_amp @ u_V)
            c.V(comp["name"], comp["pos"], comp["neg"], f"dc 0 ac {ac_amp}")
            #src.ac_magnitude = ac_amp @ u_V  
            
        elif k == "SUBCKT":
            subckt_class = subckt_registry[comp["model"]]
            sc = subckt_class()
            c.subcircuit(sc)
            c.X(comp["instance"], comp["model"], comp["n1"], comp["n2"], comp["n3"])
    return c

In [97]:
def add_subckt(nodes, ir,instance):
    instance = instance
    ir["components"].append({
    "kind": "SUBCKT",
    "instance" : instance,
    "model" : "rc_lp",
    "n1" : nodes[0],
    "n2" : nodes[1],
    "n3" : nodes[2]
    })

In [98]:
def build(nodes):
    n = 1
    ir = empty_ir("candidate")
    add_vin_and_load(ir, ac=1.0, Rload=1e3)
    for i in nodes:
        add_subckt(i,ir, n)
        n += 1
    return ir

In [99]:
#run loop once, add node, run again
def enum_cir(steps):
    all_circuits = []
    nodes = ["0", "in", "out"]
    for i in range(1, steps+1):
        if(i>1):
            nodes.append(f"n{i-1}")
        node_pairs = itertools.permutations(nodes, 3)
        for ops in itertools.product(node_pairs, repeat=i):
                ir = build(ops)
                ckt = emit_pyspice(ir, SUBCKT_REGISTRY)
                all_circuits.append({"ir":ir, "circuit": ckt})
    return all_circuits

In [100]:
cands = enum_cir(steps=3) 

In [101]:
len(cands)

216582

In [102]:
k = cands[-1]
print(k["circuit"])

.title candidate
.subckt rc_lp n1 n2 n3
R1 n1 n2 1kOhm
C1 n2 n3 1uF
.ends rc_lp
Vin in 0 dc 0 ac 1.0
RRpath in out 0.001
RRload out 0 1000.0
X1 n2 n1 out rc_lp
X2 n2 n1 out rc_lp
X3 n2 n1 out rc_lp

